## Adventures with Elephants

Description of the dataset:
---------------------------

This dataset has been recorded Angela S. Stoeger (angela.stoeger-horwath@univie.ac.at) from Vienna University and colleagues. Acoustic recordings were collected from three female and two male African elephants (Loxodonta africana) aged between 9 and 17 years located at Adventures with Elephants, Bela Bela, South Africa in 2011. The elephants were fully habituated to human presence and free to roam around in a savannah reserve of 300 ha. Vocalizations were recorded during two social contexts: spatial separation and subsequent bondings, recording distances between the elephant and the microphone ranged from 10 to several hundreds of meters. 
In 359 minutes of recordings we annotated 635 rumbles by manually tagging the beginning and the end of each rumble.

In [13]:
# annotations are in matlab file, which apparently is a hdf5 format
import scipy.io as sio

# source label annotations
filepath = '../../data/AdventuresWithElephants_Dataset/anno/groundTruth.mat'

# source location wav files
source_path = '../../data/AdventuresWithElephants_Dataset/wavs/all'
# target path
#dest_path = "../../data/AdventuresWithElephants_Dataset/wavs/samples/"
dest_path = "../../data/audio/serval-data/raw/elephants-angela"

annos = sio.loadmat(filepath)
data = annos['variable']

In [3]:
# r=[(x['SHAPE@XY'][i][0],x['SHAPE@XY'][i][1],x['PrimeKey'][i]) for i in range(x.shape[0])]
# unpack nd array
u = [(data[i,0][0],data[i,1][0][0],data[i,2][0][0],data[i,3][0]) for i in range(1,data.shape[0])]

In [4]:
# build dataframe from list
import pandas as pd
df = pd.DataFrame(data=u)  # 1st row as the column names
df.columns=['filename','start','duration','type']

Data format:
------------

Folder "wavs" contains all recorded signals (sampled at 44100Hz, 16bit, mono)

Folder "anno" contains the annotation in binary Matlab format (*.mat). The annotations include:

- groundTruth.mat: a cell that contains one annotated rumble in each row. The first column provides the name of the wav-file. The second column gives the start position of the rumble in samples; the third column contains the length of the annotated rumble in samples. The fourth column contains the type of rumble. There are two types (the usual "rumble" and the "snumble", which is a mixture of snort and rumble).

In [6]:
df['type'].value_counts()

rumble     607
snumble     11
Name: type, dtype: int64

In [5]:
# issue with EFAF2011A011.WAV en 012.WAV
df = df[df['filename'] != "EFAF2011A011.WAV"]
#df = df[df['filename'] != "EFAF2011A012.WAV"]
df = df.reset_index(drop=True)

In [7]:
# Code die aangeeft of de 10 seconden ervoor background is (1=ja, 0=nee)
df["background"] = 1
sr_target = 44100
# als (start-sr > 0) dan 0
df.loc[df["start"] < sr_target, "background"] = 0

In [8]:
for i in range(0, df.shape[0]):
    # skip 1e rij
    if(i==0):
        df.loc[i, "background"] = 0
    else:
        # Bereken eindtijd van vorige rij
        previous_end = df.loc[i-1, "start"] + df.loc[i-1, "duration"]
        # Check of filename zelfde is als file vorige rij
        if(df.loc[i-1, "filename"] == df.loc[i, "filename"]):
            # Check of previous end + sr_target*10 > start (dan 0)
            if(previous_end + sr_target*10 > df.loc[i, "start"]):
                #df.at[i,'ifor']
                df.loc[i, "background"] = 0
        else:
            df.loc[i, "background"] = 0

In [9]:
df['background'].value_counts()

0    312
1    306
Name: background, dtype: int64

In [10]:
df.iloc[210:213,]

,filename,start,duration,type,background
210,EFAF2011A022.WAV,19995019,217364,rumble,1
211,EFAF2011A022.WAV,21145760,166217,rumble,1
212,EFAF2011A022.WAV,23240082,161622,rumble,1


In [15]:
# now we open the source file, cut out the piece that is annotated and save that as a 5 second sample
import sys, csv, os, shutil
import numpy as np

# we will also resample the original recordings
#import librosa
import soundfile as sf
import resampy

from tqdm import tqdm

# this is the sample reate we want
#sr_target = 44100
sr_target = 16000

fileList = []

# for each annotation do
for index,row in tqdm(df.iterrows(), total=len(df)):
    print("index: " + str(index))
    print("row: ")
    print(row)
    # map mat file contents
    source_filename = row.filename
    start = row.start
    duration = row.duration
    category = row.type
    print("source filename: " + source_filename)
    # read source wav 
    src_file_path = os.path.normcase("%s/%s" % (source_path,source_filename))
    print(src_file_path)
    # check if sourcefile exists
    if os.path.exists(src_file_path):
        # resample
        # Load audio file at its native sampling rate
        #x, sr_orig = librosa.load(src_file_path, mono=True, sr=None, dtype=np.int16)
        x, sr_orig = sf.read(src_file_path, dtype='int16')
        print("x: ")
        print(x)
        print("sr_orig: " + str(sr_orig))
        # We can resample this to any sampling rate we like, say 16000 Hz
        # define 5 seconds
        delta = 10 * sr_orig
        # clip from start to next 5 seconds resample in target sample rate
        y = resampy.resample(x[start:start + delta + 1], sr_orig, sr_target)
        print("y: ")
        print(y)
        # check if previous 5 secs is background
        # if so create a background sample
        if (row.background == 1):
            # clip from start to next 5 seconds resample in target sample rate
            yb = resampy.resample(x[start-delta:start+1], sr_orig, sr_target)
            print("yb class: " + str(yb))
            # make destination path
            b_dst_fname = "Background" + source_filename.split('.')[0] + '-' + str(round((start-delta)/sr_orig)) + '.' + source_filename.split('.')[1]
            # add meta data to list
            fileList.append([b_dst_fname,'background'])
            # path
            b_dst_file_path = "%s/%s" % (dest_path,b_dst_fname)
            # write it back
            #librosa.output.write_wav(b_dst_file_path, yb, sr_target)
            sf.write(b_dst_file_path, yb, sr_target)
        # make destination path
        dest_filename = "Elephant" + source_filename.split('.')[0] + '-' + str(round(start/sr_orig)) + '.' + source_filename.split('.')[1]
        dst_file_path = "%s/%s" % (dest_path,dest_filename)

        # create meta data file
        fileList.append([dest_filename,category])

        # write it back
        #librosa.output.write_wav(dst_file_path, y, sr_target)
        sf.write(dst_file_path, y, sr_target)
        #break;
    else:
        print('File %s not found.' % source_filename)
        break;

  0%|          | 0/618 [00:00<?, ?it/s]

index: 0
row: 
filename      EFAF2011A008.WAV
start                  1688633
duration                 51548
type                    rumble
background                   0
Name: 0, dtype: object
source filename: EFAF2011A008.WAV
../../data/AdventuresWithElephants_Dataset/wavs/all/EFAF2011A008.WAV
x: 
[-53 -16 -15 ...  76  72  59]
sr_orig: 44100


  0%|          | 1/618 [00:02<24:32,  2.39s/it]

y: 
[   0    0    0 ... -458 -404 -584]
index: 1
row: 
filename      EFAF2011A008.WAV
start                  1829197
duration                 24136
type                    rumble
background                   0
Name: 1, dtype: object
source filename: EFAF2011A008.WAV
../../data/AdventuresWithElephants_Dataset/wavs/all/EFAF2011A008.WAV
x: 
[-53 -16 -15 ...  76  72  59]
sr_orig: 44100


  0%|          | 2/618 [00:03<15:45,  1.54s/it]

...

y: 
[  0   0   0 ... 709 636 662]
index: 34
row: 
filename      EFAF2011A010.WAV
start                 10790405
duration                 30363
type                    rumble
background                   0
Name: 34, dtype: object
source filename: EFAF2011A010.WAV
../../data/AdventuresWithElephants_Dataset/wavs/all/EFAF2011A010.WAV
x: 
[-18 -31 -52 ... -17 -18 -39]
sr_orig: 44100


  6%|▌         | 35/618 [00:32<08:56,  1.09it/s]

y: 
[  0   0   0 ... -19 -11 -17]
index: 35
row: 
filename      EFAF2011A010.WAV
start                 13432313
duration                196077
type                    rumble
background                   1
Name: 35, dtype: object
source filename: EFAF2011A010.WAV
../../data/AdventuresWithElephants_Dataset/wavs/all/EFAF2011A010.WAV
x: 
[-18 -31 -52 ... -17 -18 -39]
sr_orig: 44100
y: 
[-141 -355 -398 ...   -5   -6   -3]


  6%|▌         | 36/618 [00:33<09:00,  1.08it/s]

yb class: [   0    0    0 ...  -82  -73 -190]
index: 36
row: 
filename      EFAF2011A012.WAV
start                  4167674
duration                 37754
type                    rumble
background                   0
Name: 36, dtype: object
source filename: EFAF2011A012.WAV
../../data/AdventuresWithElephants_Dataset/wavs/all/EFAF2011A012.WAV
File EFAF2011A012.WAV not found.


In [16]:
# save the metadata for later
fileList
md = pd.DataFrame(fileList, columns=["filename", "category"])  
md.to_csv('./labels/custom/jungle/meta-data-elephant-adventures.csv',index=False)

In [17]:
md.tail()

,filename,category
40,ElephantEFAF2011A010-239.WAV,rumble
41,ElephantEFAF2011A010-242.WAV,rumble
42,ElephantEFAF2011A010-245.WAV,rumble
43,BackgroundEFAF2011A010-295.WAV,background
44,ElephantEFAF2011A010-305.WAV,rumble


In [18]:
# copy all wavs to audio dir in serval
import glob
import shutil
import os

src_dir = "../../data/audio/serval-data/raw/elephants-angela"
dst_dir = "../../data/audio/serval-data/wav-16bit-16khz/elephants-angela"


for jpgfile in glob.iglob(os.path.join(src_dir, "*.WAV")):
    shutil.copy(jpgfile, dst_dir)
